In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch Dataset 

In [2]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

#data_path = "../dataset/TSmall"
#postfix = "TSmall"
#dim = 70
data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(index))

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_triu_norm, mu, std = AbstractDataset.normalize(s_triu)


s_train, p_train, s_test, p_test = split(s_triu_norm, p_triu, ind)

In [3]:
s_train.shape, p_train.shape, s_test.shape, p_test.shape

((800, 351), (800, 351), (201, 351), (201, 351))

# Fetch Model

In [4]:
filepath = "../../models/" + "TSmall_sto3gmodelTSmall_sto3g_250-150-50+triu.h5"
model = keras.models.load_model(filepath)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 351)               123552    
_________________________________________________________________
dense_1 (Dense)              (None, 601)               211552    
_________________________________________________________________
dense_2 (Dense)              (None, 501)               301602    
_________________________________________________________________
dense_3 (Dense)              (None, 401)               201302    
_________________________________________________________________
dense_4 (Dense)              (None, 351)               141102    
Total params: 979,110
Trainable params: 979,110
Non-trainable params: 0
_________________________________________________________________


#  Analysis

In [5]:
p_nn = model.predict(s_test)

In [6]:
from SCFInitialGuess.utilities.dataset import StaticDataset

dataset = StaticDataset(
    train=(s_train, p_train),
    validation=(None, None),
    test=(s_test, p_test),
    mu=mu,
    std=std
)

In [7]:
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities

N_ELECTRONS = 30


## Pure 

In [8]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_nn,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=True,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-11-06 00:12:22: Iteration calculation: 0
[ ] 2018-11-06 00:12:22: Iteration calculation: 1
[ ] 2018-11-06 00:12:23: Iteration calculation: 2
[ ] 2018-11-06 00:12:23: Iteration calculation: 3
[ ] 2018-11-06 00:12:23: Iteration calculation: 4
[ ] 2018-11-06 00:12:23: Iteration calculation: 5
[ ] 2018-11-06 00:12:23: Iteration calculation: 6
[ ] 2018-11-06 00:12:23: Iteration calculation: 7
[ ] 2018-11-06 00:12:24: Iteration calculation: 8
[ ] 2018-11-06 00:12:24: Iteration calculation: 9
[ ] 2018-11-06 00:12:24: Iteration calculation: 10
[ ] 2018-11-06 00:12:24: Iteration calculation: 11
[ ] 2018-11-06 00:12:24: Iteration calculation: 12
[ ] 2018-11-06 00:12:25: Iteration calculation: 13
[ ] 2018-11-06 00:12:26: Iteration calculation: 14
[ ] 2018-11-06 00:12:26: Iteration calculation: 15
[ ] 2018-11-06 00:12:26: Iteration calculation: 16
[ ] 2018-11-06 00:12:27: Iteration calculation: 17
[ ] 2018-11-06 00:12:27: Iteration calculation: 18
[ ] 2018-11-06 00:12:27: Iteration calcul

[ ] 2018-11-06 00:13:22: Iteration calculation: 160
[ ] 2018-11-06 00:13:22: Iteration calculation: 161
[ ] 2018-11-06 00:13:22: Iteration calculation: 162
[ ] 2018-11-06 00:13:23: Iteration calculation: 163
[ ] 2018-11-06 00:13:23: Iteration calculation: 164
[ ] 2018-11-06 00:13:23: Iteration calculation: 165
[ ] 2018-11-06 00:13:23: Iteration calculation: 166
[ ] 2018-11-06 00:13:23: Iteration calculation: 167
[ ] 2018-11-06 00:13:24: Iteration calculation: 168
[ ] 2018-11-06 00:13:24: Iteration calculation: 169
[ ] 2018-11-06 00:13:24: Iteration calculation: 170
[ ] 2018-11-06 00:13:24: Iteration calculation: 171
[ ] 2018-11-06 00:13:24: Iteration calculation: 172
[ ] 2018-11-06 00:13:25: Iteration calculation: 173
[ ] 2018-11-06 00:13:25: Iteration calculation: 174
[ ] 2018-11-06 00:13:25: Iteration calculation: 175
[ ] 2018-11-06 00:13:26: Iteration calculation: 176
[ ] 2018-11-06 00:13:26: Iteration calculation: 177
[ ] 2018-11-06 00:13:26: Iteration calculation: 178
[ ] 2018-11-

In [12]:
10 / 201

0.04975124378109453

## Damping 

In [9]:
def format_results(result):
    if isinstance(result, list):
        out = list(map(
            lambda x: "{:0.5E} +- {:0.5E}".format(*x),
            result
        ))
        out = "\n".join(out)
    else:
        out =  "{:0.5E} +- {:0.5E}".format(*result)
    return out

In [10]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics
from SCFInitialGuess.utilities.dataset import make_matrix_batch

print("--- Iterations Damped ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
    make_matrix_batch(p_nn, dim, is_triu=True).astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-06 00:13:33: Iteration calculation: 0
[ ] 2018-11-06 00:13:33: Iteration calculation: 1
[ ] 2018-11-06 00:13:33: Iteration calculation: 2
[ ] 2018-11-06 00:13:33: Iteration calculation: 3
[ ] 2018-11-06 00:13:33: Iteration calculation: 4
[ ] 2018-11-06 00:13:33: Iteration calculation: 5
[ ] 2018-11-06 00:13:34: Iteration calculation: 6
[ ] 2018-11-06 00:13:34: Iteration calculation: 7
[ ] 2018-11-06 00:13:34: Iteration calculation: 8
[ ] 2018-11-06 00:13:34: Iteration calculation: 9
[ ] 2018-11-06 00:13:34: Iteration calculation: 10
[ ] 2018-11-06 00:13:35: Iteration calculation: 11
[ ] 2018-11-06 00:13:35: Iteration calculation: 12
[ ] 2018-11-06 00:13:35: Iteration calculation: 13
[ ] 2018-11-06 00:13:35: Iteration calculation: 14
[ ] 2018-11-06 00:13:36: Iteration calculation: 15
[ ] 2018-11-06 00:13:36: Iteration calculation: 16
[ ] 2018-11-06 00:13:36: Iteration calculation: 17
[ ] 2018-11-06 00:13:36: Iteration calculation: 18
[ ] 2018-11-06 00:13:37: Iteration calcul

[ ] 2018-11-06 00:14:09: Iteration calculation: 160
[ ] 2018-11-06 00:14:09: Iteration calculation: 161
[ ] 2018-11-06 00:14:10: Iteration calculation: 162
[ ] 2018-11-06 00:14:10: Iteration calculation: 163
[ ] 2018-11-06 00:14:10: Iteration calculation: 164
[ ] 2018-11-06 00:14:10: Iteration calculation: 165
[ ] 2018-11-06 00:14:10: Iteration calculation: 166
[ ] 2018-11-06 00:14:10: Iteration calculation: 167
[ ] 2018-11-06 00:14:11: Iteration calculation: 168
[ ] 2018-11-06 00:14:11: Iteration calculation: 169
[ ] 2018-11-06 00:14:11: Iteration calculation: 170
[ ] 2018-11-06 00:14:11: Iteration calculation: 171
[ ] 2018-11-06 00:14:11: Iteration calculation: 172
[ ] 2018-11-06 00:14:11: Iteration calculation: 173
[ ] 2018-11-06 00:14:11: Iteration calculation: 174
[ ] 2018-11-06 00:14:12: Iteration calculation: 175
[ ] 2018-11-06 00:14:12: Iteration calculation: 176
[ ] 2018-11-06 00:14:12: Iteration calculation: 177
[ ] 2018-11-06 00:14:12: Iteration calculation: 178
[ ] 2018-11-

## DIIS 

In [11]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics
from SCFInitialGuess.utilities.dataset import make_matrix_batch

print("--- Iterations Damped ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    make_matrix_batch(p_nn, dim, is_triu=True).astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-06 00:14:18: Iteration calculation: 0
[ ] 2018-11-06 00:14:18: Iteration calculation: 1
[ ] 2018-11-06 00:14:19: Iteration calculation: 2
[ ] 2018-11-06 00:14:19: Iteration calculation: 3
[ ] 2018-11-06 00:14:19: Iteration calculation: 4
[ ] 2018-11-06 00:14:19: Iteration calculation: 5
[ ] 2018-11-06 00:14:19: Iteration calculation: 6
[ ] 2018-11-06 00:14:19: Iteration calculation: 7
[ ] 2018-11-06 00:14:19: Iteration calculation: 8
[ ] 2018-11-06 00:14:20: Iteration calculation: 9
[ ] 2018-11-06 00:14:20: Iteration calculation: 10
[ ] 2018-11-06 00:14:20: Iteration calculation: 11
[ ] 2018-11-06 00:14:20: Iteration calculation: 12
[ ] 2018-11-06 00:14:20: Iteration calculation: 13
[ ] 2018-11-06 00:14:21: Iteration calculation: 14
[ ] 2018-11-06 00:14:21: Iteration calculation: 15
[ ] 2018-11-06 00:14:21: Iteration calculation: 16
[ ] 2018-11-06 00:14:21: Iteration calculation: 17
[ ] 2018-11-06 00:14:21: Iteration calculation: 18
[ ] 2018-11-06 00:14:21: Iteration calcul

[ ] 2018-11-06 00:14:49: Iteration calculation: 160
[ ] 2018-11-06 00:14:49: Iteration calculation: 161
[ ] 2018-11-06 00:14:50: Iteration calculation: 162
[ ] 2018-11-06 00:14:50: Iteration calculation: 163
[ ] 2018-11-06 00:14:50: Iteration calculation: 164
[ ] 2018-11-06 00:14:50: Iteration calculation: 165
[ ] 2018-11-06 00:14:51: Iteration calculation: 166
[ ] 2018-11-06 00:14:51: Iteration calculation: 167
[ ] 2018-11-06 00:14:51: Iteration calculation: 168
[ ] 2018-11-06 00:14:51: Iteration calculation: 169
[ ] 2018-11-06 00:14:52: Iteration calculation: 170
[ ] 2018-11-06 00:14:52: Iteration calculation: 171
[ ] 2018-11-06 00:14:52: Iteration calculation: 172
[ ] 2018-11-06 00:14:52: Iteration calculation: 173
[ ] 2018-11-06 00:14:52: Iteration calculation: 174
[ ] 2018-11-06 00:14:53: Iteration calculation: 175
[ ] 2018-11-06 00:14:53: Iteration calculation: 176
[ ] 2018-11-06 00:14:53: Iteration calculation: 177
[ ] 2018-11-06 00:14:53: Iteration calculation: 178
[ ] 2018-11-